# Peer-graded Assignment: 
## Segmenting and Clustering Neighborhoods in Toronto

### part 1: 
based on information on wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M a pandas dataframe with columns
    * PostalCode
    * Borough
    * Neighborhood
   

The wikipedia list is a list of postal codes in Canada where the first letter is M. 
Postal codes beginning with M are located within the city of Toronto in the province of Ontario.

### documentation
https://beautiful-soup-4.readthedocs.io/en/latest/#

 see https://www.youtube.com/watch?v=ng2o98k983k for use of beautifullsoup
    

In [17]:
# with open('simple.html') as html_file:
#    soup = BeautifulSoup(html_file, 'lxml')
# print(soup) ## the entire code, not indented   
# print(soup.prettify()) ## indented

# match = soup.title  ## finds the title tag (including tags)
# match = soup.title.text
# print(match)

# match = soup.div
# match = soup.find('div')   ## results the same (13:00)
# match = soup.find('div', class_='footer')  ## finds a certain (here underscore because class is python specific word

# article = soup.find('div', class_='article')
# headline = article.h2.a.text
# summary = article.p.text

# for article in soup.find_all('div', class_='article'):
#    headline = article.h2.a.text
#    summary = article.p.text


### libraries install


In [18]:
## in anaconda prompt:  
## conda install -c conda-forge beautifulsoup4 --yes
## conda install -c conda-forge lxml --yes   # 4:33
## conda install -c conda-forge html5lib --yes   # not done
##
## conda install -c conda-forge geocoder --yes   # not done, too much changes in the environment
## conda install -c esri arcgis --verbose


### libraries import


In [19]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np

from arcgis.gis import GIS
#from arcgis.gis import *
from arcgis.geocoding import geocode, reverse_geocode
#from arcgis.geometry import Point

### data scraping

In [20]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup_raw = BeautifulSoup(source, 'html5')
# print(soup_raw.prettify())
print("done")

done


In [21]:
soup_table = soup_raw.tbody
# print(soup_table)
## parser has added columnnames into tbody while wikipediasite had a thead-tag
print("done")

done


In [22]:
## building the dataframe from the soup_table result
scraped_result = []
#result = pd.DataFrame(columns = ["PostalCode"])
scraped_result = pd.DataFrame(columns = ["PostalCode","Borough","Neighborhood"])
for soup_tr in soup_table.find_all('tr'):
    i = 1 
    ## for empty records in tablerows where there is no table data (like the table header line) a dummy result is given
    PostalCode = "A1A"
    Borough = "Not assigned"
    Neighborhood = "Not assigned"
    for  soup_td in soup_tr.find_all('td'):
        if i == 3:
            Neighborhood = str(soup_td.text)
            Neighborhood = Neighborhood.replace("\n","")
        if i == 2:
            Borough = str(soup_td.text)
            i = 3
        if i == 1:
            PostalCode = str(soup_td.text)
            i = 2
    newline = pd.DataFrame([[PostalCode,Borough,Neighborhood]], columns = ["PostalCode","Borough","Neighborhood"] )
    scraped_result = scraped_result.append(newline, ignore_index=True)
print('done')
# scraped_result
#newline

done


### cleaning data

***Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.***

In [23]:
## removing lines with "Not assigned" in Borough, this includes eventual dummy records
cleaning1 = scraped_result.copy()
cleaning1 = cleaning1.loc[cleaning1['Borough'] != "Not assigned"]
print('done')
# cleaning1

done


***If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
So, the value of the Borough AND the Neighborhood columns will be 'Queen's Park'***

In [24]:
## where "not assigned" in Neighborhood copy the name of the borough
## assumption is made that this never applies for PostalCodes which cover multiple Neighborhoods, 
## hence the order of cleaning can be different than in the 'rules' of the assignment 
cleaning2 = cleaning1.copy()
## splitting the dataset in two parts, not to be adjusted, and to adjusted
cleaning2a = cleaning2.loc[cleaning2['Neighborhood'] != "Not assigned"]
cleaning2b = cleaning2.loc[cleaning2['Neighborhood'] == "Not assigned"]
cleaning2b['Neighborhood'] = cleaning2b['Borough'] 
## previous line induced the following error: 
##     "A value is trying to be set on a copy of a slice from a DataFrame.
##     Try using .loc[row_indexer,col_indexer] = value instead
##     See the caveats in the documentation: 
##     http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy"
## In this case it is a false positive: 
## see https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas#20627316

## and combining the two parts
cleaning2 = cleaning2a.append(cleaning2b)
print('done')
# cleaning2


done


F:\programfilesF\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


***More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.***

In [25]:
## the dataframe cleaning3 is buildup by adding lines from previous step (cleaning2)
cleaning3 = pd.DataFrame(columns = ["PostalCode", "Borough", "Neighborhood"])
for postalcode in set(cleaning2['PostalCode']):
    tempstring = ''
    no_of_neighborhoods = cleaning2[cleaning2['PostalCode'] == postalcode].shape[0]
    ## making a small dataframe with just the current postalcode
    tempdata = cleaning2[cleaning2['PostalCode'] == postalcode]
    tempdata = tempdata.reset_index()
    for i in range(0,no_of_neighborhoods):
        ## the assumption is made that each PostalCode, even when it compasses more than one Neighborhood
        ## it only compasses only one Borough. so it doesn't matter from which line the borough information comes
        borough = tempdata.loc[i,'Borough']
        strippednb = tempdata.loc[i,'Neighborhood'].strip()
        tempstring = tempstring + strippednb 
        if i < no_of_neighborhoods-1:
            tempstring = tempstring + ", "
    newline = pd.DataFrame([[postalcode,borough,tempstring]], columns = ["PostalCode","Borough","Neighborhood"] )
    cleaning3 = cleaning3.append(newline)

cleaning3 = cleaning3.reset_index()
print('done')
#cleaning3 


done


In [26]:
resultA = cleaning3.copy()

## results

In [27]:
#resultA

In [28]:
resultA.shape

(103, 4)

## Getting geo-information
since installing of geocoder required a substantial change to my local environment including DOWNGRADES  of packages I opted for using ArcGIS environment. geocoding and reverse geocoding can be done anonymous.


### libraries install 
for a good overview of the notebook installing libraries is done in the preamble of this notebook.

    * from arcgis.gis import GIS
    * #from arcgis.gis import#
    * from arcgis.geocoding import geocode, reverse_geocode
    * #from arcgis.geometry import Point

## retrieving the information
getting information about neighborhoods is done in a series of calls to the api. 
I will try if the first 3 (alphanumerics) of the PostalCode will yield an answer

Log into ArcGIS Online as an anonymous user.
Simple geocoding and reverse geocoding do not require credits, hence you need not log in using your credentials.

In [29]:
dev_gis = GIS()

In [30]:
#geocode(address="Bathorst Manor")[0]
#geocode(address ="M3H", search_extent=None, location=None, distance=None, out_sr=None, category=None, out_fields='*', max_locations=10, magic_key=None, for_storage=False, geocoder=None, as_featureset=False)

In [31]:
geodataframe = resultA.copy()
geodataframe['Latitude'] = 0
geodataframe['Longitude'] = 0


In [34]:
for i in range(geodataframe.shape[0]):
    postalcode = geodataframe.loc[i,'PostalCode']
    queryresult = geocode(address=postalcode,search_extent=None, out_fields='location', max_locations=1, as_featureset=False)
    longitude = queryresult[0]['location']['x'] 
    latitude = queryresult[0]['location']['y']
    geodataframe.loc[i, 'Longitude'] = longitude
    geodataframe.loc[i, 'Latitude'] = latitude

In [35]:
resultB=geodataframe.copy()

## Results

In [36]:
resultB

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180
1,0,M1S,Scarborough,Agincourt,43.793940,-79.267976
2,0,M9W,Etobicoke,Northwest,43.711740,-79.579410
3,0,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.648690,-79.385440
4,0,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.648573,-79.578250
5,0,M4J,East York,East Toronto,43.688765,-79.334175
6,0,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675
7,0,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.625070,-79.527135
8,0,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.609920,-79.498049
9,0,M3M,North York,Downsview Central,43.733610,-79.496750
